In [1]:
import numpy as np

lambda_rent_1 = 3
lambda_rent_2 = 4
lambda_return_1 = 3
lambda_return_2 = 2
car_max = 20
move_max = 5
reward_per_car = 10
move_cost_per_car = 2
gamma = 0.9

In [2]:
state_space = [(i, j) for i in range(car_max + 1) for j in range(car_max + 1)]

def poisson_probability(lmbda, k):
    return (lmbda ** k) * np.exp(-lmbda) / np.math.factorial(k)

def reward(cars_1, cars_2, rental_1, rental_2, move_cost):
    revenue = reward_per_car * (min(cars_1, rental_1) + min(cars_2, rental_2))
    return revenue - move_cost

In [3]:
def value_iteration():
    V = np.zeros((car_max + 1, car_max + 1))
    policy = np.zeros((car_max + 1, car_max + 1), dtype=int)
    for iteration in range(500):
        V_new = np.copy(V)
        for i in range(car_max + 1):
            for j in range(car_max + 1):
                max_value = -float('inf')
                best_action = 0
                for move in range(-move_max, move_max + 1):
                    if 0 <= i - move <= car_max and 0 <= j + move <= car_max:
                        rental_1 = np.random.poisson(lambda_rent_1)
                        rental_2 = np.random.poisson(lambda_rent_2)
                        return_1 = np.random.poisson(lambda_return_1)
                        return_2 = np.random.poisson(lambda_return_2)
                        next_cars_1 = min(i - move + return_1, car_max)
                        next_cars_2 = min(j + move + return_2, car_max)
                        move_cost = abs(move) * move_cost_per_car
                        total_reward = reward(i, j, rental_1, rental_2, move_cost)
                        total_value = total_reward + gamma * V[next_cars_1, next_cars_2]
                        if total_value > max_value:
                            max_value = total_value
                            best_action = move
                V_new[i, j] = max_value
                policy[i, j] = best_action
        if np.max(np.abs(V_new - V)) < 1e-6:
            break
        V = V_new
    return V, policy

In [4]:
V, policy = value_iteration()
print("Optimal Value Function (V):")
print(V)
print("\nOptimal Policy (Action to move cars):")
print(policy)
i, j = 14, 12
print(f"Optimal action for state (i={i}, j={j}): Move {policy[i, j]} cars.")

Optimal Value Function (V):
[[ 795.81505787  872.19507787  877.83035902  893.55635902  910.08235902
   925.17635902  930.63507787  933.49479587  930.08235902  970.45105787
   913.0848916   926.63507787  939.9219758   885.4422916   907.8848916
   860.95697878  885.36861928  921.1019758   891.1019758   876.96861928
   867.7939758 ]
 [ 857.73635902  883.55635902  910.08235902  919.46375902  918.95635902
   934.86375902  942.86375902  941.49479587  931.26375902  935.26375902
   919.8848916   941.58707787  951.50707787  872.5482916   917.69584878
   927.3022916   857.5022916   891.5548916   875.2422916   860.16861928
   868.96861928]
 [ 884.73505787  891.55635902  918.08235902  918.95635902  941.17635902
   946.83707787  926.86375902  982.86375902  980.86375902  917.69584878
   957.50707787  951.4422916   939.50707787  922.78707787  974.1939758
   917.3022916   878.3394338   866.16861928  865.2222916   877.2673876
   891.2583758 ]
 [ 889.83035902  916.08235902  946.86375902  951.26375902  9